In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,year,school_name,reading_score,maths_score,School ID,type,size,budget
0,0,Paul Bradley,M,9,Huang High School,96,94,0,Government,2917,1910635
1,1,Victor Smith,M,12,Huang High School,90,43,0,Government,2917,1910635
2,2,Kevin Rodriguez,M,12,Huang High School,41,76,0,Government,2917,1910635
3,3,Richard Scott,M,12,Huang High School,89,86,0,Government,2917,1910635
4,4,Bonnie Ray,F,9,Huang High School,87,69,0,Government,2917,1910635


In [2]:
# Calculate the Totals (Schools and Students)
school_count = len(school_data_complete["school_name"].unique())
student_count = school_data_complete["Student ID"].count()

# Calculate the Total Budget
total_budget = school_data["budget"].sum()

In [3]:
# Calculate the average maths score
average_math_score = school_data_complete["maths_score"].mean()
average_reading_score = school_data_complete["reading_score"].mean()


In [4]:
# Calculate the average reading score
reading_average = school_data_complete["reading_score"].mean()
reading_average

69.98013786060761

In [5]:
# Calculate the percentage of students with a passing maths score (50 or greater)
#count the number of students with a grade of over 50 & find the percentage based on the total number of students
passing_math_count = school_data_complete[(school_data_complete["maths_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100

passing_math_reading_count = school_data_complete[(school_data_complete["maths_score"] >= 70) 
                                                  & (school_data_complete["reading_score"] >= 70)].count()["student_name"]

overall_passing_rate = passing_math_reading_count /  float(student_count) * 100

In [6]:
# Minor Data Cleanup
district_summary = pd.DataFrame({"Total Schools": [school_count], 
                                 "Total Students": [student_count], 
                                 "Total Budget": [total_budget],
                                 "Average Math Score": [average_math_score], 
                                 "Average Reading Score": [average_reading_score],
                                 "% Passing Math": [passing_math_percentage],
                                 "% Passing Reading": [passing_reading_percentage],
                                 "% Overall Passing": [overall_passing_rate]})

district_summary = district_summary[["Total Schools", "Total Students", "Total Budget",
                                     "Average Math Score", 
                                     "Average Reading Score",
                                     "% Passing Math",
                                     "% Passing Reading",
                                     "% Overall Passing"]]

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",70.338192,69.980138,51.460301,50.814399,26.316058


In [14]:
school_types = school_data.set_index(["school_name"])["type"]

# Calculate the total student count
per_school_counts = school_data_complete["school_name"].value_counts()

# Calculate the total school budget and per capita spending
# per_school_budget = school_data_complete.groupby(["school_name"]).mean()["budget"]
per_school_budget = school_data_complete.groupby(["school_name"]).mean()["budget"]
per_school_capita = per_school_budget / per_school_counts

# Calculate the average test scores
per_school_math = school_data_complete.groupby(["school_name"]).mean()["maths_score"]
per_school_reading = school_data_complete.groupby(["school_name"]).mean()["reading_score"]

In [8]:
# Get the students who passed math and passed reading by creating separate filtered DataFrames.
school_passing_math = school_data_complete[(school_data_complete["maths_score"] >= 70)]
school_passing_reading = school_data_complete[(school_data_complete["reading_score"] >= 70)]

# Get the the students who passed both reading and math in a separate DataFrame.
passing_math_and_reading = school_data_complete[(school_data_complete["reading_score"] >= 70)
                                               & (school_data_complete["maths_score"] >= 70)]

In [9]:
per_school_passing_math = school_passing_math.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
per_school_passing_reading = school_passing_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
overall_passing_rate = passing_math_and_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100

In [10]:
# Convert to DataFrame
per_school_summary = pd.DataFrame({"School Type": school_types,
                                   "Total Students": per_school_counts,
                                   "Total School Budget": per_school_budget,
                                   "Per Student Budget": per_school_capita,
                                   "Average Math Score": per_school_math,
                                   "Average Reading Score": per_school_reading,
                                   "% Passing Math": per_school_passing_math,
                                   "% Passing Reading": per_school_passing_reading,
                                   "% Overall Passing": overall_passing_rate})

# Minor data munging
per_school_summary = per_school_summary[["School Type", "Total Students", "Total School Budget", "Per Student Budget",
                                         "Average Math Score", "Average Reading Score", 
                                         "% Passing Math", "% Passing Reading", 
                                         "% Overall Passing"]]
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,55.225080,52.451768,29.159968
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,53.175457,53.283100,28.579117
Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,47.677179,49.169210,23.736860
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,48.959474,50.492881,24.497992
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,54.700272,54.155313,31.675749
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,49.255663,49.471413,24.660194
Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,57.142857,55.503513,30.913349
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,48.920123,49.777168,23.723003
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,48.771267,48.603235,23.881537
Pena High School,Independent,962,"$585,858.00",$609.00,72.088358,71.613306,55.405405,55.093555,30.457380


In [11]:
# Sort and show top five schools
top_schools = per_school_summary.sort_values(["% Overall Passing"], ascending=False)
top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,54.700272,54.155313,31.675749
Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,57.142857,55.503513,30.913349
Pena High School,Independent,962,"$585,858.00",$609.00,72.088358,71.613306,55.405405,55.093555,30.457380
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,55.225080,52.451768,29.159968
Rodriguez High School,Government,3999,"$2,547,363.00",$637.00,72.047762,70.935984,54.513628,52.788197,28.832208


In [12]:
# Sort and show bottom five schools
bottom_schools = per_school_summary.sort_values(["% Overall Passing"], ascending=True)
bottom_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,48.920123,49.777168,23.723003
Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,47.677179,49.169210,23.736860
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,48.771267,48.603235,23.881537
Wilson High School,Independent,2283,"$1,319,574.00",$578.00,69.170828,68.876916,49.364871,48.313622,24.485326
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,48.959474,50.492881,24.497992


In [13]:
# Create data series of scores by grade levels using conditionals
ninth_graders = school_data_complete[(school_data_complete["year"] == 9)]
tenth_graders = school_data_complete[(school_data_complete["year"] == 10)]
eleventh_graders = school_data_complete[(school_data_complete["year"] == 11)]
twelfth_graders = school_data_complete[(school_data_complete["year"] == 12)]

# Group each by school name
ninth_graders_scores = ninth_graders.groupby(["school_name"]).mean()["maths_score"]
tenth_graders_scores = tenth_graders.groupby(["school_name"]).mean()["maths_score"]
eleventh_graders_scores = eleventh_graders.groupby(["school_name"]).mean()["maths_score"]
twelfth_graders_scores = twelfth_graders.groupby(["school_name"]).mean()["maths_score"]

# Combine series into single DataFrame
scores_by_grade = pd.DataFrame({"9th": ninth_graders_scores, "10th": tenth_graders_scores,
                                "11th": eleventh_graders_scores, "12th": twelfth_graders_scores})

# Minor data munging
scores_by_grade = scores_by_grade[["9th", "10th", "11th", "12th"]]
scores_by_grade.index.name = None

# Display the DataFrame
scores_by_grade

,9th,10th,11th,12th
Bailey High School,72.493827,71.897498,72.374900,72.675097
Cabrera High School,72.321970,72.437768,71.008299,70.604712
Figueroa High School,68.477804,68.331586,68.811001,69.325282
Ford High School,69.021609,69.387006,69.248862,68.617811
Griffin High School,72.789731,71.093596,71.692521,71.469178
Hernandez High School,68.586831,68.867156,69.154412,68.985075
Holden High School,70.543307,75.105263,71.640777,73.409639
Huang High School,69.081754,68.533246,69.431345,68.639316
Johnson High School,69.469286,67.990220,68.637730,69.287393
Pena High School,71.996364,72.396000,72.523438,71.187845
